# Imports and Setup

In [20]:
# Importing Necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Data Preparation

In [21]:
trainpath = '/kaggle/input/riceleafdiseasedataset/dataset/train'
testpath = '/kaggle/input/riceleafdiseasedataset/dataset/test'

# Image Processing

In [22]:
# Image Processing - Training Data
new_size = 224
train_images = []
train_labels = []
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

for i in os.listdir(trainpath):
    if i in class_disease:
        print("Entering the folder:", i)
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            train_images.append(list(resize_image))
            train_labels.append(class_disease[i])

# Image Processing - Testing Data
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):
    if i in class_disease:
        print("Entering to the folder name:", i)
        files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(testpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder is", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            test_images.append(list(resize_image))
            test_labels.append(class_disease[i])

def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

gpus = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(gpus)}")

if len(gpus) < 2:
    print("Not enough GPUs available, ensure your environment is configured correctly")
else:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.MirroredStrategy()

Entering the folder: Tungro
Number of images in the folder: 1108
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BrownSpot
Number of images in the folder: 1400
Entering to the folder name: Tungro
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BrownSpot
Number of images in the folder is 200
(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)
Num GPUs Available: 2


# Model Definition - ResNet50

In [23]:
# Deep Feature Extraction - ResNet50
def model_resnet50():
    Resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in Resnet_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(Resnet_model.output)
    output = Model(inputs=Resnet_model.input, outputs=feature)
    return output

# Training and Evaluation - Random Forest

In [24]:
# Random Forest Classifier - VGG19 Deep Features
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)

        rf = RandomForestClassifier()
        rf = rf.fit(train_feature_50, y_train)
        test_pred = rf.predict(test_feature_50)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step
Run 1 - Accuracy: 0.96250000, Recall: 0.96250000, Precision: 0.99873096, F1 Score: 0.97999179, AUC: 0.98104167
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
Run 2 - Accuracy: 0.97125000, Recall: 0.97125000, Precision: 0.99871134, F1 Score: 0.98472248, AUC: 0.98541667
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
Run 3 - Accuracy: 0.97250000, Recall: 0.97250000, Precision: 1.00000000, F1 Score: 0.98597167, AUC: 0.98625000
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
Run 4 - Accuracy: 0.97125000, Recall: 0.97125000, Precision: 1.00000000, F1 Score: 0.98527923, AUC: 0.98562500
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
Run 5 - Accuracy: 0.96625000, Recall: 0.96625000, Precision: 1.00000000, F1 Score: 0.98258702, AUC: 0.98312500
161/161 ━━━━━━━━━━━━━━━━━━━━ 1

# Training and Evaluation - KNN

In [27]:
from sklearn.neighbors import KNeighborsClassifier

# KNN Classifier - ResNet50 Deep Features
def train_and_evaluate_model(X_train, y_train, X_test, y_test, run_seed):
    np.random.seed(run_seed)
    tf.random.set_seed(run_seed)
    
    with strategy.scope():
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)

        knn = KNeighborsClassifier()
        knn.fit(train_feature_50, y_train)
        test_pred = knn.predict(test_feature_50)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    run_seed = i  # Set a different seed for each run
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test, run_seed)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")


161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 1 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 0.99875622, F1 Score: 0.99874999, AUC: 0.99916667
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 2 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 0.99875622, F1 Score: 0.99874999, AUC: 0.99916667
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 3 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 0.99875622, F1 Score: 0.99874999, AUC: 0.99916667
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 4 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 0.99875622, F1 Score: 0.99874999, AUC: 0.99916667
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 90ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 5 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 0.99875622, F1 Score: 0.99874999, AUC: 0.99916667
161/161 ━━━━━━━━━━━━━━━━━━━━ 1

# Training and Evaluation - SVM

In [28]:
# SVM Classifier - VGG19 Deep Features
def train_and_evaluate_model(X_train, y_train, X_test, y_test, run_seed):
    
    np.random.seed(run_seed)
    tf.random.set_seed(run_seed)
    
    with strategy.scope():
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        svm = SVC(probability=True)
        svm.fit(train_feature_50, y_train_cat)
        test_pred = svm.predict(test_feature_50)
        test_pred_proba = svm.predict_proba(test_feature_50)

        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    run_seed = i
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test, run_seed)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 1 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
Run 2 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 3 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
Run 4 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
Run 5 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 1

# Training and Evaluation - Decision Tree Classifier

In [29]:
from sklearn.tree import DecisionTreeClassifier

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)

        # Define and train Decision Tree Classifier
        dt = DecisionTreeClassifier()
        dt.fit(train_feature_50, y_train)
        test_pred = dt.predict(test_feature_50)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")


161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 1 - Accuracy: 0.90875000, Recall: 0.90875000, Precision: 0.90935689, F1 Score: 0.90783407, AUC: 0.93916667
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 2 - Accuracy: 0.90500000, Recall: 0.90500000, Precision: 0.90447469, F1 Score: 0.90418636, AUC: 0.93666667
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
Run 3 - Accuracy: 0.91125000, Recall: 0.91125000, Precision: 0.91138391, F1 Score: 0.91058494, AUC: 0.94083333
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
Run 4 - Accuracy: 0.90375000, Recall: 0.90375000, Precision: 0.90314565, F1 Score: 0.90198556, AUC: 0.93583333
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 5 - Accuracy: 0.90250000, Recall: 0.90250000, Precision: 0.90247259, F1 Score: 0.90162140, AUC: 0.93500000
161/161 ━━━━━━━━━━━━━━━━━━━━ 2

# Training and Evaluation - XGBoost Classifier

In [30]:
from xgboost import XGBClassifier

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        # Define and train XGBoost Classifier
        xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        xgb.fit(train_feature_50, y_train_cat)
        test_pred = xgb.predict(test_feature_50)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, xgb.predict_proba(test_feature_50), multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")


161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 1 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 0.99999583
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
Run 2 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 0.99999583
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 3 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 0.99999583
161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 4 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 0.99999583
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 5 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 0.99999583
161/161 ━━━━━━━━━━━━━━━━━━━━ 1

# Results and Metrics